writing mimic

Set variables, import deps, give model something to write about, and tell it how many times to iteratively improve writing mimic

In [ ]:
import os
import anthropic
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Set up Anthropic API client
api_key = "YOUR_API_KEY"  # replace this, clod
client = anthropic.Client(api_key)

# Set sampling parameters
model = "claude-3-sonnet-20240229"
temperature = 0.7
top_p = 1

# Load sample texts from directory
samples_dir = "samples/"
sample_texts = []
for file_name in os.listdir(samples_dir):
    file_path = os.path.join(samples_dir, file_name)
    with open(file_path, "r") as file:
        sample_texts.append(file.read())

topic = input("Enter the topic or information to write about: ")  # Prompt user for information to write about
num_iterations = int(input("Enter the number of iterations to perform: "))  # Set number of iterations

first mimicked text

In [ ]:
prompt = f"Analyze the following sample texts to understand the writing style:\n\n{sample_texts}\n\nNow, write a text about the following topic in the same style as the sample texts:\n{topic}"
response = client.completions.create(
    prompt=prompt,
    model=model,
    temperature=temperature,
    top_p=top_p,
    max_tokens=500
)
mimicked_text = response.completions[0].text.strip()
print(mimicked_text)

calculate similarity score and improve text generation

In [ ]:
def calculate_similarity(sample_texts, mimicked_text):
    # Preprocess texts
    texts = sample_texts + [mimicked_text]
    processed_texts = [re.sub(r'\W+', ' ', text.lower()) for text in texts]
   
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)
   
    # Calculate cosine similarity
    similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    average_similarity = np.mean(similarity_scores)
   
    return average_similarity

similarity_score = calculate_similarity(sample_texts, mimicked_text)

# Iteratively improve mimicked text
for i in range(num_iterations):
    print(f"Iteration {i+1}")
    print("Similarity Score:", similarity_score)
   
    if similarity_score >= 0.85:
        print("Desired similarity score achieved.")
        break
   
    prompt = f"Here is the current mimicked text:\n{mimicked_text}\n\nThe similarity score compared to the sample texts is {similarity_score:.2f}. Improve the mimicked text by focusing on capturing the key stylistic elements of the sample texts more accurately."
    response = client.completions.create(
        prompt=prompt,
        model=model,
        temperature=temperature,
        top_p=top_p,
        max_tokens=500
    )
    mimicked_text = response.completions[0].text.strip()
    similarity_score = calculate_similarity(sample_texts, mimicked_text)

# Provide explanation of similarity score calculation
explanation = "The similarity score was calculated using the following steps:\n"
explanation += "1. Preprocess the sample texts and mimicked text by removing non-word characters and converting to lowercase.\n"
explanation += "2. Create a TF-IDF vectorizer to convert the texts into numerical feature vectors.\n"
explanation += "3. Calculate the cosine similarity between the mimicked text's feature vector and each sample text's feature vector.\n"
explanation += "4. Take the average of the cosine similarity scores to obtain the final similarity score."
print("<similarity_score_calculation>")
print(explanation)
print("</similarity_score_calculation>")
print("<similarity_score>")
print(f"{similarity_score:.2f}")
print("</similarity_score>")

save text to file

In [ ]:
# Save final mimicked text to file
output_dir = "outputs/"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "mimicked_text.txt")
with open(output_file, "w") as file:
    file.write(mimicked_text)
print("Mimicked text saved to", output_file)